In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salary2025software/2025-yazilim-sektoru-maaslari-onceki-yazilimci.xlsx


In [2]:
df = pd.read_excel("/kaggle/input/salary2025software/2025-yazilim-sektoru-maaslari-onceki-yazilimci.xlsx")

df

,level,position,tech_stack,experience,gender,company,company_size,work_type,city,currency,salary,raise_period
0,Senior,Back-end Developer,"Php, NodeJS, Go",15 Yıl ve üzeri,Erkek,E-Ticaret,101 - 249 Kişi,Remote,* İngiltere,$ - Dolar,8.000 - 8.999,1
1,Senior,Full Stack Developer,"Java, Angular",5 - 7 Yıl,Erkek,Sigorta,250+,Remote,Ankara,₺ - Türk Lirası,140.000 - 144.999,2
2,Senior,Mobile Application Developer (iOS),"Swift, Objective C",10 - 12 Yıl,Erkek,Banka,250+,Remote,Muğla,₺ - Türk Lirası,195.000 - 199.999,2
3,Senior,Mobile Application Developer (iOS),Swift,7 - 10 Yıl,Erkek,E-Ticaret,250+,Ofis,İstanbul,₺ - Türk Lirası,220.000 - 229.999,1
4,Middle,Full Stack Developer,Php,1 - 3 Yıl,Erkek,E-Ticaret,1 - 5 Kişi,Ofis,İstanbul,₺ - Türk Lirası,35.000 - 35.999,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9051,Senior,Team / Tech Lead,".Net, NodeJS, Delphi",15 Yıl ve üzeri,Erkek,ERP & CRM,101 - 249 Kişi,Hibrit (Ofis + Remote),İzmir,₺ - Türk Lirası,175.000 - 179.999,1
9052,Senior,Team / Tech Lead,".Net, Python, Angular",10 - 12 Yıl,Erkek,Yazılım Evi & Danışmanlık,1 - 5 Kişi,Remote,* Birleşik Arap Emirlikleri,₺ - Türk Lirası,170.000 - 174.999,1
9053,Junior,Back-end Developer,Java,0 - 1 Yıl,Erkek,Telekomünikasyon,21 - 50 Kişi,Hibrit (Ofis + Remote),* Azerbaycan,₺ - Türk Lirası,12.000 - 12.999,1
9054,Middle,Front-end Developer,"JavaScript | Html | Css, Vue",1 - 3 Yıl,Erkek,E-Ticaret,101 - 249 Kişi,Hibrit (Ofis + Remote),Kahramanmaraş,₺ - Türk Lirası,64.000 - 64.999,1


In [3]:
print(df.head())

    level                            position          tech_stack  \
0  Senior                  Back-end Developer     Php, NodeJS, Go   
1  Senior                Full Stack Developer       Java, Angular   
2  Senior  Mobile Application Developer (iOS)  Swift, Objective C   
3  Senior  Mobile Application Developer (iOS)               Swift   
4  Middle                Full Stack Developer                 Php   

        experience gender    company    company_size work_type         city  \
0  15 Yıl ve üzeri  Erkek  E-Ticaret  101 - 249 Kişi    Remote  * İngiltere   
1        5 - 7 Yıl  Erkek    Sigorta            250+    Remote       Ankara   
2      10 - 12 Yıl  Erkek      Banka            250+    Remote        Muğla   
3       7 - 10 Yıl  Erkek  E-Ticaret            250+      Ofis     İstanbul   
4        1 - 3 Yıl  Erkek  E-Ticaret      1 - 5 Kişi      Ofis     İstanbul   

          currency             salary  raise_period  
0        $ - Dolar      8.000 - 8.999             1  
1 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9056 entries, 0 to 9055
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   level         9056 non-null   object
 1   position      9056 non-null   object
 2   tech_stack    9052 non-null   object
 3   experience    9056 non-null   object
 4   gender        9056 non-null   object
 5   company       9056 non-null   object
 6   company_size  9056 non-null   object
 7   work_type     9056 non-null   object
 8   city          9056 non-null   object
 9   currency      9056 non-null   object
 10  salary        9056 non-null   object
 11  raise_period  9056 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 849.1+ KB


**SATIR VE SÜTUN KONTROLÜ**

In [5]:
satir, sütun = df.shape
print(f"Satır sayısı(veri sayısı): {satir} \nDeğişken sayısı: {sütun}")

Satır sayısı(veri sayısı): 9056 
Değişken sayısı: 12


In [6]:
df.describe()

,raise_period
count,9056.000000
mean,1.538759
std,0.538767
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,4.000000


**NULL VERİ VE UNİQUE KONTROLÜ**

In [7]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"unique {column} sayaç {len(unique_values)}")
    print(f"uniques {df[column].unique()}")
    print()
    boş_veri = df[column].isnull().sum()

    if boş_veri > 0:
        print(f"Null veri var! Null veri sayısı: {boş_veri}")
    else:
        print("Null veri yoktur!")
    print()
    
    print("****")


unique level sayaç 3
uniques ['Senior' 'Middle' 'Junior']

Null veri yoktur!

****
unique position sayaç 59
uniques ['Back-end Developer' 'Full Stack Developer'
 'Mobile Application Developer (iOS)' 'QA / Manuel Test'
 'Front-end Developer' 'Team / Tech Lead' 'QA / Automation'
 'Data Scientist' 'Director of Software Development' 'Game Developer'
 'Mobile Application Developer (Full Stack & Cross)' 'Data Analyst'
 'Embedded Software Developer' 'Product Owner' 'Software Architect'
 'Mobile Application Developer (Android)' 'Business Analyst'
 'Product Manager' 'DevOps Engineer' 'Consultant'
 'Software Development Manager / Engineering Manager' 'Project Manager'
 'Software Engineer' 'RPA Developer' 'UI/UX Designer' 'AI Engineer'
 'Database Administrator (DBA)' 'Data Engineer' 'Chief Data Officer'
 'IT Manager' 'CTO' 'Robotic Software Engineer' 'R&D Manager'
 'System Engineer' 'Support Engineer' 'Cyber Security' 'Network Engineer'
 'SAP / ABAP Developer & Consultant' 'Business Intelligence'

**TEKRARLAYAN SATIR KONTROLU**

In [8]:
tekrarlayan_kayit=df[df.duplicated()]
print(f"tekrarlayan veri sayısı {tekrarlayan_kayit.shape[0]}")
print(f"tekrarlayan veriler {tekrarlayan_kayit}")

tekrarlayan veri sayısı 30
tekrarlayan veriler        level                                position  \
585   Junior                    Full Stack Developer   
1543  Senior                      Back-end Developer   
2332  Senior                      Back-end Developer   
2422  Middle             Embedded Software Developer   
3219  Middle                        Business Analyst   
3822  Senior                     Front-end Developer   
3823  Senior                     Front-end Developer   
3824  Senior                     Front-end Developer   
3891  Middle                     Front-end Developer   
4154  Middle             Embedded Software Developer   
4456  Senior                      Back-end Developer   
4746  Middle                      Back-end Developer   
5164  Senior                      Software Architect   
5293  Middle                    Full Stack Developer   
5312  Senior                      Back-end Developer   
5801  Middle                    Full Stack Developer   
5

**TEKRARLAYAN SATIRLARIN SİLİNMESİ**

In [9]:
df.drop_duplicates(inplace=True)

**TEKRARLAYAN SATIRLARIN SİLİNMESİ SONRASI SATIR SAYISI**

In [10]:
satir, sütun = df.shape
print(f"Satır sayısı(veri sayısı): {satir} \nDeğişken sayısı: {sütun}")

Satır sayısı(veri sayısı): 9026 
Değişken sayısı: 12


**tech_stack eksik verilerini doldurma**

In [11]:
doldur = "belirtilmemis"
df['tech_stack'].fillna(doldur,inplace=True)


boş_veri = df['tech_stack'].isnull().sum()
print(f"Tech_stack boş veri kontrolü : {boş_veri}")

Tech_stack boş veri kontrolü : 0


/tmp/ipykernel_13/1849050509.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tech_stack'].fillna(doldur,inplace=True)
